In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csv
from enum import Enum

from loguru import logger
from pydantic import BaseModel

from taskllm.optimizer.data import DataSet, Row
from taskllm.optimizer.methods import BayesianTrainer
from taskllm.optimizer.prompt.meta import PromptMode

# logger.remove()  # remove the old handler. Else, the old one will work (and continue printing DEBUG logs) along with the new handler added below'
# logger.add(sys.stdout, level="TRACE")  # add a new handler which has INFO as the default


class Ratings(Enum):
    ONE = "1"
    TWO = "2"
    THREE = "3"
    FOUR = "4"
    FIVE = "5"
    NA = "N/A"


class StarbucksReviewRating(BaseModel):
    rating: Ratings


def sentiment_scoring_function(
    row: Row[StarbucksReviewRating], output: StarbucksReviewRating | None
) -> float:
    if output is None:
        return -10
    if not row.expected_output:
        return 0
    logger.trace(f"Expected: {row.expected_output.rating}, Output: {output.rating}")
    if row.expected_output.rating == output.rating:
        return 1

    return 0


def load_file_as_dataset(path: str) -> DataSet:
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            rating = StarbucksReviewRating(rating=Ratings(row["Rating"]))
            rows.append(
                Row.create(
                    input_dictionary={
                        "review": row["Review"],
                        "name": row["name"],
                        "location": row["location"],
                        "date": row["Date"],
                    },
                    output=rating,
                )
            )
    return DataSet(rows=rows[:50], name="starbucks_reviews")



/Users/bill/dev/taskllm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
csv_path = "./starbucks_reviews.csv"
dataset = load_file_as_dataset(csv_path)
trainer = BayesianTrainer(
    all_rows=dataset,
    task_guidance="determine the rating of this review",
    keys=["review", "name", "location", "date"],
    expected_output_type=StarbucksReviewRating,
    scoring_function=sentiment_scoring_function,
    num_iterations=4,  # Start with fewer iterations for testing
    candidates_per_iteration=4,  # Start with fewer candidates for testing
    prompt_mode=PromptMode.SIMPLE,
    models=[
        "anthropic/claude-3-haiku-20240307",
        "openai/gpt-4.1-nano-2025-04-14",
        "openai/gpt-4.1-mini-2025-04-14",
        "groq/gemma2-9b-it",
        "groq/qwen-qwq-32b"
    ],
    failure_analysis_enabled=True
)

2025-05-12 21:36:41.894 | INFO     | taskllm.optimizer.methods.bayesian:__init__:208 - Using CPU for Pyro/Torch computations
2025-05-12 21:36:41.894 | INFO     | taskllm.optimizer.methods.base:__init__:263 - All rows: 50
2025-05-12 21:36:41.895 | DEBUG    | taskllm.optimizer.methods.bayesian:__init__:727 - BayesianTrainer initialized


In [8]:
await trainer.train()

2025-05-12 21:36:43.601 | INFO     | taskllm.optimizer.methods.bayesian:train:4 - Starting Bayesian optimization with Pyro: 4 iterations, 4 candidates/iter.
2025-05-12 21:36:43.602 | INFO     | taskllm.optimizer.methods.bayesian:train:5 - Phase 1: Generating and evaluating initial candidates...
2025-05-12 21:36:43.602 | INFO     | taskllm.optimizer.prompt.meta:generate_spec:139 - Generating simple prompt content for: determine the rating of this review

Use plain language in the prompt you write...
2025-05-12 21:36:45.793 | INFO     | taskllm.optimizer.prompt.meta:generate_spec:139 - Generating simple prompt content for: determine the rating of this review

Make the prompt you write as simple as possible...


As a review analysis expert, your role is to determine the overall rating of a customer review. Carefully read the review provided and assess the sentiments expressed. Consider the key points mentioned by the reviewer to decide how positive or negative the review is. Based on your assessment, assign a rating from 1 to 5, where 1 is very negative and 5 is very positive. Here's the specific context for your analysis:

- Review: {{ review }}
- Name of reviewer: {{ name }}
- Location of reviewer: {{ location }}
- Date of review: {{ date }}


2025-05-12 21:36:47.339 | INFO     | taskllm.optimizer.prompt.meta:generate_spec:139 - Generating simple prompt content for: determine the rating of this review

Make the prompt you write as simple as possible...


You are a review analyst. Determine the rating for the following review based on its sentiment. Consider the tone, keywords, and overall context to assign a rating from 1 to 5. Review details:

Review: {{ review }}
Reviewer Name: {{ name }}
Location: {{ location }}
Date: {{ date }}


2025-05-12 21:36:48.360 | INFO     | taskllm.optimizer.prompt.meta:generate_spec:139 - Generating simple prompt content for: determine the rating of this review

Use plain language in the prompt you write...


...

2025-05-12 21:46:53.283 | DEBUG    | taskllm.optimizer.methods.bayesian:fit_surrogate_model:312 - SVI iteration 100/1000, Loss: 22.3861
2025-05-12 21:46:53.350 | DEBUG    | taskllm.optimizer.methods.bayesian:fit_surrogate_model:312 - SVI iteration 200/1000, Loss: 22.3857
2025-05-12 21:46:53.416 | DEBUG    | taskllm.optimizer.methods.bayesian:fit_surrogate_model:312 - SVI iteration 300/1000, Loss: 22.3855
2025-05-12 21:46:53.480 | DEBUG    | taskllm.optimizer.methods.bayesian:fit_surrogate_model:312 - SVI iteration 400/1000, Loss: 22.3854
2025-05-12 21:46:53.543 | DEBUG    | taskllm.optimizer.methods.bayesian:fit_surrogate_model:312 - SVI iteration 500/1000, Loss: 22.3853
2025-05-12 21:46:53.605 | DEBUG    | taskllm.optimizer.methods.bayesian:fit_surrogate_model:312 - SVI iteration 600/1000, Loss: 22.3853
2025-05-12 21:46:53.667 | DEBUG    | taskllm.optimizer.methods.bayesian:fit_surrogate_model:312 - SVI iteration 700/1000, Loss: 22.3852
2025-05-12 21:46:53.729 | DEBUG    | taskllm.opt


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers



2025-05-12 21:46:54.747 | DEBUG    | taskllm.optimizer.methods.base:get_scores:91 - Calculated 40 scores
2025-05-12 21:46:54.747 | INFO     | taskllm.optimizer.methods.base:calculate_scores:116 - Model openai/gpt-4.1-nano-2025-04-14 with prompt achieved score: 35.0000, Correct: 35, Incorrect: 5, Unlabelled: 0 out of 40
2025-05-12 21:46:54.747 | INFO     | taskllm.optimizer.methods.bayesian:train:66 - Iteration 4: No improvement found. Best score remains 35.0000
2025-05-12 21:46:54.748 | INFO     | taskllm.optimizer.methods.bayesian:train:69 - Phase 3: Final evaluation on test set...
2025-05-12 21:46:54.748 | INFO     | taskllm.optimizer.methods.bayesian:train:75 - Added Pyro dependencies to pyproject.toml (commented)
2025-05-12 21:46:54.748 | INFO     | taskllm.optimizer.methods.bayesian:train:78 - Analyzing performance by model...
2025-05-12 21:46:54.749 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:74 - Getting outputs for 10 rows using anthropic/claude-3-haiku-20240307
202


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers



2025-05-12 21:48:43.863 | DEBUG    | taskllm.optimizer.methods.base:get_scores:91 - Calculated 10 scores
2025-05-12 21:48:43.865 | INFO     | taskllm.optimizer.methods.base:calculate_scores:116 - Model groq/qwen-qwq-32b with prompt achieved score: 8.0000, Correct: 8, Incorrect: 2, Unlabelled: 0 out of 10
2025-05-12 21:48:43.866 | DEBUG    | taskllm.optimizer.methods.base:get_scores:91 - Calculated 10 scores
2025-05-12 21:48:43.867 | INFO     | taskllm.optimizer.methods.base:calculate_scores:116 - Model groq/qwen-qwq-32b with prompt achieved score: 0.0000, Correct: 0, Incorrect: 10, Unlabelled: 0 out of 10
2025-05-12 21:48:43.867 | DEBUG    | taskllm.optimizer.methods.base:get_scores:91 - Calculated 10 scores
2025-05-12 21:48:43.869 | INFO     | taskllm.optimizer.methods.base:calculate_scores:116 - Model groq/qwen-qwq-32b with prompt achieved score: 9.0000, Correct: 9, Incorrect: 1, Unlabelled: 0 out of 10
2025-05-12 21:48:43.869 | DEBUG    | taskllm.optimizer.methods.base:get_scores:91